# Cloud Carbon Coefficients

This notebook calculates the coefficients used in [Cloud Carbon Footprint](https://www.cloudcarbonfootprint.org/), an application that estimates the energy (kilowatt hours) and carbon emissions (metric tons CO2e) from public cloud provider utilization.

These values are based on the [SPECpower_ssj2008 results](https://www.spec.org/power_ssj2008/results/). New data is released every quarter so this notebook helps calculate updated coefficients. The full results are in [data/](data/).

#### Imports

In [1]:
%pip install -r requirements.txt

import csv
import numpy as np
import pandas as pd

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Processor types

Processors are grouped into types by each vendor e.g. Intel Broadwell CPUs. Cloud providers provide the CPU types for each of their instance types, but not the precise CPU details. As such, we calculate the average wattage by processor type.

In [2]:
# Loads a CSV file then returns each row appended to an array
def load_append_data(file_name):
    with open(f'data/{file_name}', 'r') as csvfile:
        reader = csv.reader(csvfile)

        data = []
        for row in reader:
            data.append(row[0])
        
        return data

cpus_intel_sandybridge = load_append_data('intel-sandybridge.csv')
cpus_intel_ivybridge = load_append_data('intel-ivybridge.csv')
cpus_intel_haswell = load_append_data('intel-haswell.csv')
cpus_intel_broadwell = load_append_data('intel-broadwell.csv')
cpus_intel_skylake = load_append_data('intel-skylake.csv')
cpus_intel_cascadelake = load_append_data('intel-cascadelake.csv')
cpus_intel_coffeelake = load_append_data('intel-coffeelake.csv')

## Processor lists

Now we know which processors are in which type, we can group all the tested servers by their CPU type to calculate: average idle watts, average watts at 100% utilization, average GB/chip.

In [3]:
# Load all servers from SPECpower results CSV
servers = pd.read_csv('data/SPECpower-full-results.csv', na_values=['NC'])

#### Regex match

The regex to match the CPU names matches to the end of the line using `$` because some chips have version numbers after, so we can't just do a substring match e.g. `Intel E3-1230` is a Sandy Bridge chip but `Intel E3-1230 v3` is Haswell. It is case insensitive due to the use of `v3` and `V3` inconsistently in the definitions.

#### Clean data

The SPECpower results often appends extra info to the `CPU Description` column which is unecessary. For example, `Intel Xeon E5-2470 (Intel Turbo Boost Technology up to 3.10 GHz)`. This extra info needs to be stripped e.g. to `Intel Xeon E5-2470` otherwise the regex match will not work.

The check below will error if the data is not clean.

In [4]:
if len(servers[servers['CPU Description'].str.contains('Ghz')]) > 0:
    print('Data not clean')
    import sys
    sys.exit(1)

### Intel: Sandy Bridge

In [18]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_sandybridge]
servers_intel_sandybridge = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_sandybridge = {}
intel_sandybridge['idle_watts'] = (servers_intel_sandybridge['avg. watts @ active idle'].astype(float) / servers_intel_sandybridge['Total Threads']).mean()
intel_sandybridge['100% watts'] = (servers_intel_sandybridge['avg. watts @ 100%'].astype(float) / servers_intel_sandybridge['Total Threads']).mean()
intel_sandybridge['GB/Chip'] = (servers_intel_sandybridge['Total Memory (GB)'] / servers_intel_sandybridge['Chips']).mean()
intel_sandybridge

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


{'idle_watts': 2.1694411458333334,
 '100% watts': 8.575357663690477,
 'GB/Chip': 16.480916030534353}

### Intel: Ivy Bridge

In [33]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_ivybridge]
servers_intel_ivybridge = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_ivybridge = {}
intel_ivybridge['idle_watts'] = (servers_intel_ivybridge['avg. watts @ active idle'].astype(float) / servers_intel_ivybridge['Total Threads']).mean()
intel_ivybridge['100% watts'] = (servers_intel_ivybridge['avg. watts @ 100%'].astype(float) / servers_intel_ivybridge['Total Threads']).mean()
intel_ivybridge['GB/Chip'] = (servers_intel_ivybridge['Total Memory (GB)'] / servers_intel_ivybridge['Chips']).mean()
intel_ivybridge

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


{'idle_watts': 3.0369270833333335,
 '100% watts': 8.248611111111112,
 'GB/Chip': 14.933333333333334}

### Intel: Haswell

In [34]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_haswell]
servers_intel_haswell = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_haswell = {}
intel_haswell['idle_watts'] = (servers_intel_haswell['avg. watts @ active idle'].astype(float) / servers_intel_haswell['Total Threads']).mean()
intel_haswell['100% watts'] = (servers_intel_haswell['avg. watts @ 100%'].astype(float) / servers_intel_haswell['Total Threads']).mean()
intel_haswell['GB/Chip'] = (servers_intel_haswell['Total Memory (GB)'] / servers_intel_haswell['Chips']).mean()
intel_haswell

{'idle_watts': 1.0005952380952383,
 '100% watts': 4.739715608465608,
 'GB/Chip': 27.714285714285715}

### Intel: Broadwell

In [20]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_broadwell]
servers_intel_broadwell = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_broadwell = {}
intel_broadwell['idle_watts'] = (servers_intel_broadwell['avg. watts @ active idle'].astype(float) / servers_intel_broadwell['Total Threads']).mean()
intel_broadwell['100% watts'] = (servers_intel_broadwell['avg. watts @ 100%'].astype(float) / servers_intel_broadwell['Total Threads']).mean()
intel_broadwell['GB/Chip'] = (servers_intel_broadwell['Total Memory (GB)'] / servers_intel_broadwell['Chips']).mean()
intel_broadwell

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


{'idle_watts': 0.7128342245989304,
 '100% watts': 3.6853275401069516,
 'GB/Chip': 69.6470588235294}

### Intel: Sky Lake

In [28]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_skylake]
servers_intel_skylake = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_skylake = {}
intel_skylake['idle_watts'] = (servers_intel_skylake['avg. watts @ active idle'].astype(float) / servers_intel_skylake['Total Threads']).mean()
intel_skylake['100% watts'] = (servers_intel_skylake['avg. watts @ 100%'].astype(float) / servers_intel_skylake['Total Threads']).mean()
intel_skylake['GB/Chip'] = (servers_intel_skylake['Total Memory (GB)'] / servers_intel_skylake['Chips']).mean()
intel_skylake

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


{'idle_watts': 0.6522727665317579,
 '100% watts': 4.255505654535773,
 'GB/Chip': 81.32432432432432}

### Intel: Cascade Lake

In [30]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_cascadelake]
servers_intel_cascadelake = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_cascadelake = {}
intel_cascadelake['idle_watts'] = (servers_intel_cascadelake['avg. watts @ active idle'].astype(float) / servers_intel_cascadelake['Total Threads']).mean()
intel_cascadelake['100% watts'] = (servers_intel_cascadelake['avg. watts @ 100%'].astype(float) / servers_intel_cascadelake['Total Threads']).mean()
intel_cascadelake['GB/Chip'] = (servers_intel_cascadelake['Total Memory (GB)'] / servers_intel_cascadelake['Chips']).mean()
intel_cascadelake

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


{'idle_watts': 0.6389493581523519,
 '100% watts': 3.9673047343937564,
 'GB/Chip': 98.11764705882354}

### Intel: Coffee Lake

In [23]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_coffeelake]
servers_intel_coffeelake = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_coffeelake = {}
intel_coffeelake['idle_watts'] = (servers_intel_coffeelake['avg. watts @ active idle'].astype(float) / servers_intel_coffeelake['Total Threads']).mean()
intel_coffeelake['100% watts'] = (servers_intel_coffeelake['avg. watts @ 100%'].astype(float) / servers_intel_coffeelake['Total Threads']).mean()
intel_coffeelake['GB/Chip'] = (servers_intel_coffeelake['Total Memory (GB)'] / servers_intel_coffeelake['Chips']).mean()
intel_coffeelake

{'idle_watts': 1.138425925925926,
 '100% watts': 5.421759259259258,
 'GB/Chip': 19.555555555555557}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2bb85156-92ac-4c34-9a24-94671c8f593c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>